<a href="https://colab.research.google.com/github/PaulaDCV/Scraping_Job_Vacancies/blob/main/Job_Vacancies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests
!pip install BeautifulSoup4
!pip install scrapfly-sdk


In [33]:
from getpass import getpass
secret = getpass('Enter the secret value: ')

Enter the secret value: ··········


In [127]:
import requests
from bs4 import BeautifulSoup
import csv
import datetime
import time
from scrapfly import ScrapflyClient, ScrapeConfig
import pandas as pd

# Indeed

## Scraping

In [194]:
def url_generation_indeed(position, location):
  url = "https://uk.indeed.com/jobs?q="+position+"&l="+location
  return url

In [195]:
def request_listings_indeed(url):
  client = ScrapflyClient(key=secret)
  result = client.scrape(ScrapeConfig(
      url= url,
      asp=True,
      # ^ enable Anti Scraping Protection
  ))
  html_text = result.content
  soup = BeautifulSoup(html_text, "html5lib")
  return soup

In [196]:
url = url_generation_indeed("data","london")
print("URL:"+url)
soup = request_listings_indeed(url)


URL:https://uk.indeed.com/jobs?q=data&l=london


### unused code

In [45]:
def request_listings_old(url):
  #Change User Agent to avoid being blocked
  headers = {
      "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
  }
  html_text = requests.get(url, headers = headers).text
  soup = BeautifulSoup(html_text, "html5lib")
  return soup

## Processing Step

In [219]:
#Extract individual info of each job
def extract_job_info_indeed(job):
  jobs = []
  title_element = job.find("h2", class_="jobTitle")
  jobs.append( title_element.text)

  href = title_element.find("a",attrs= {"role": "button"}).get("href","Not Available")
  jobs.append(href)

  location = job.find("div", attrs={"data-testid": "text-location" })
  jobs.append(location.text)

  company = job.find("span", attrs ={ "data-testid" : "company-name"} )
  jobs.append(company.text)

  jobs.append("indeed")
  return jobs

In [220]:
#Extract every job posting into a list
def extract_jobs_indeed(results):
  #Create dataframe
  df = pd.DataFrame(columns = ['Title', 'Url', 'Location','Company', 'Site'])
  #Get Jobs
  jobs = results.find_all("td", class_="resultContent")
  #Extract info of each job
  for job in jobs:
    job_processed = extract_job_info_indeed(job)

    df.loc[len(df.index)] = job_processed
  return df

In [ ]:
extract_jobs_indeed(soup)

In [ ]:
def run_indeed():
  url = url_generation_indeed("data","london")
  print("URL:"+url)
  soup = request_listings_indeed(url)
  if len(soup)<0:
    print("Indeed scraped")
    df = extract_jobs_indeed(soup)
    print("Indeed jobs scraped")
    return df

# Glassdoor




1.   https://www.glassdoor.com/Job/london-data-scientist-jobs-SRCH_IL.0,6_IC2671300_KO7,21.htm?includeNoSalaryJobs=true
2.   https://www.glassdoor.com/Job/brighton-england-uk-data-scientist-jobs-SRCH_IL.0,19_IC2680716_KO20,34.htm



* Location: You can specify a location using the &location= parameter. For example, &location=New+York would search for jobs in New York.

* Job Title/Keyword: You can specify a job title or keyword using the &sc.keyword= parameter. For example, &sc.keyword=software+engineer would search for software engineering jobs.

* Company: You can specify a company using the &sc.company= parameter. For example, &sc.company=Google would search for jobs at Google.

* Job Type: You can specify the type of job using the &jobType= parameter. For example, &jobType=fulltime would search for full-time jobs.



In [ ]:
def url_generation_glassdoor(position, location):
  url = "https://www.glassdoor.com/Job/index.htm?&location="+location+"k&sc.keyword="+position
  return url

In [225]:
def request_listings_glassdoor(url):
  client = ScrapflyClient(key=secret)
  result = client.scrape(ScrapeConfig(
      url= url,
      asp=True,
      # ^ enable Anti Scraping Protection
  ))
  html_text = result.content
  soup = BeautifulSoup(html_text, "html5lib")
  return soup

In [239]:
url = url_generation_glassdoor("data","England")
soup = request_listings_glassdoor(url)
print(soup)


<!DOCTYPE html>
<html class="fonts-loaded-initial fonts-loaded-full" lang="en" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraph.org/schema/"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><title>Recommended Jobs For You | Glassdoor Job Search</title><meta content="Glassdoor" property="og:site_name"/><meta content="website" property="og:type"/><meta content="https://www.glassdoor.com/assets/images/meta/logo-og.png" property="og:image"/><meta content="Recommended Jobs For You | Glassdoor Job Search" property="og:title"/><meta content="Search millions of open jobs with Glassdoor job search. Research company ratings, reviews and salaries. Glassdoor helps you find a job you love." property="og:description"/><meta content="https://www.glassdoor.com/Job/index.htm" property="og:url"/><meta content="Search millions of open jobs with Glassdoor job search. Research company ratings, reviews and salaries. Glassdoor helps you find a job you lo

In [227]:
#Extract individual info of each job

GlassDoorData = {"Job Postings":["ul","class","job"],
                 "Title":["a","id","job-title"],
                 "Href":["a",""],
                 "Location":["div","id","job-location"],
                 "Company":["span","class","employer-Name"]

                 }

def extract_job_info_glassdoor(job):
  jobs = []
  title_element = job.find("h2", class_="jobTitle")
  jobs.append( title_element.text)
  jobs.append(title_element.href)

  location = job.find("div", attrs={"id": "job-location" })
  jobs.append(location.text)

  company = job.find("span", attrs ={ "class" : "employer-Name"} )
  jobs.append(company.text)

  jobs.append("Indeed")
  return jobs

In [237]:
#Extract every job posting into a list
def extract_jobs_glassdoor(results):
  #Create dataframe
  df = pd.DataFrame(columns = ['Title', 'Url', 'Location','Company', 'Site'])
  #Get Jobs
  jobs = results.find_all("ul", class_="JobsList")
  #Extract info of each job
  for job in jobs:
    print(jobs)
    job_processed = extract_job_info_glassdoor(job)
    df.loc[len(df.index)] = job_processed
  return df

In [238]:
extract_jobs_glassdoor(soup)

,Title,Url,Location,Company,Site


# CSV File

In [172]:
def main(position, location):

  indeed = run_indeed(position,location)
  glassdoor  = run_glassdoor(position,location)

  jobs_processed = indeed + glassdoor
  jobs_processed.to_csv(location+"_"+position+"_search.csv",index=False)

main("data","london")